In [1]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import gradio as gr
import pandas as pd
import numpy as np
import joblib
import shap
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [4]:
tech_survey_path = "/content/drive/MyDrive/survey.csv"
depression_anxiety_path = "/content/drive/MyDrive/scores.csv"
who_database_path = "/content/drive/MyDrive/mental.csv"


In [5]:
def prepare_datasets():
    np.random.seed(42)
    tech_survey = pd.DataFrame({
        'Symptom': ['fatigue', 'restlessness', 'lack of sleep', 'anxiety', 'depression']
    })

    depression_anxiety = pd.DataFrame({
        'Symptom': np.random.choice(['fatigue', 'restlessness', 'lack of sleep', 'anxiety', 'depression'], 100),
        'Condition': np.random.choice(['Anxiety', 'Depression', 'Stress'], 100)
    })

    who_data = pd.DataFrame({
        'Symptom': ['fatigue', 'restlessness', 'lack of sleep', 'anxiety', 'depression']
    })

    return tech_survey, depression_anxiety, who_data

In [6]:
try:
    tech_survey = pd.read_csv(tech_survey_path)
    depression_anxiety = pd.read_csv(depression_anxiety_path)
    who_data = pd.read_csv(who_database_path)
except Exception:
    print("Using alternative datasets")
    tech_survey, depression_anxiety, who_data = prepare_datasets()

In [7]:
symptoms_list = sorted(list(set(
    tech_survey['Symptom'].tolist() +
    depression_anxiety['Symptom'].tolist() +
    who_data['Symptom'].tolist()
)))

In [8]:
def prepare_model_data(symptoms_list):
    X, y = [], []
    for condition in ['Anxiety', 'Depression', 'Stress']:
        for _ in range(50):
            sample_symptoms = np.random.choice(symptoms_list, np.random.randint(1, 4), replace=False)
            features = [1 if symptom in sample_symptoms else 0 for symptom in symptoms_list]
            X.append(features)
            y.append(condition)
    return np.array(X), np.array(y)

In [9]:
X, y = prepare_model_data(symptoms_list)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

RandomForestClassifier(random_state=42)

In [10]:
joblib.dump(model, "mental_health_model.pkl")

['mental_health_model.pkl']

In [11]:
def predict_mental_health(user_input):
    try:

        if not user_input or len(user_input) > 500:
            return "Invalid input. Please provide a valid symptom description.", "", ""


        suspicious_patterns = ['script', '<', '>', 'javascript', 'alert']
        if any(pattern in user_input.lower() for pattern in suspicious_patterns):
            return "Error: Suspicious input detected.", "", ""


        normalized_input = ' '.join(user_input.lower().split())


        input_features = [1 if symptom.lower() in normalized_input else 0 for symptom in symptoms_list]


        if sum(input_features) == 0:
            return "No recognizable symptoms found. Please describe your symptoms more clearly.", "", ""

        prediction = model.predict([input_features])[0]

        coping_mechanisms = {
            "Anxiety": "Try mindfulness exercises and deep breathing techniques.",
            "Depression": "Consider journaling your thoughts and seeking therapy.",
            "Stress": "Take breaks, practice relaxation, and seek support from loved ones.",
        }
        suggestion = coping_mechanisms.get(prediction, "No specific suggestions available.")

        return prediction, f"Prediction based on symptoms.", suggestion

    except Exception as e:

        return f"Prediction error: {str(e)}", "", ""


In [12]:
with gr.Blocks() as interface:
    with gr.Row():
        gr.Markdown(
            """
            ## 🧠 Self-Analysis Mental Health Chatbot
            Welcome! Enter your symptoms, and this tool will predict possible mental health conditions, provide explanations, and suggest coping mechanisms. 💜
            """
        )
    with gr.Row():
        user_input = gr.Textbox(
            label="💬 Describe your symptoms (comma-separated)",
            placeholder="E.g., fatigue, restlessness, lack of sleep",
            lines=2,
        )
    with gr.Row():
        analyze_button = gr.Button("🔍 Analyze")

    condition_output = gr.Textbox(label="Predicted Mental Health Condition")
    explanation_output = gr.Textbox(label="Explanation of Prediction")
    suggestion_output = gr.Textbox(label="Suggested Coping Mechanism")

    analyze_button.click(
        fn=predict_mental_health,
        inputs=[user_input],
        outputs=[condition_output, explanation_output, suggestion_output],
    )

    with gr.Row():
        gr.Markdown(
            """
            ---
            ### ⚠️ Important Note:
            This chatbot provides general mental health advice and insights. For personalized medical advice, please consult a licensed mental health professional.
            """
        )

In [13]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9bf31817bcc2a44774.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
